In [114]:
import numpy as np
import pandas as pd

dataset='dataset_size.csv'
dataset_vmav='dataset_vmaf.csv'
csv_path = '../ns-3-allinone/ns-3.35/contrib/etri_mvdash/'

#### Read bitrate

In [118]:
f = open(csv_path+dataset)
mvinfo = [int(x) for x in f.readline().split()] 
f.close()

nViewpoints = mvinfo[0]
nSegments = mvinfo[1]
segDuration = mvinfo[2]
nRates = mvinfo[3]

df = pd.read_csv(csv_path+dataset,sep="\t", header=None, skiprows=1)
df_rate_kbps = pd.DataFrame(df.mean().values.reshape((nViewpoints,nRates)).transpose()*8/segDuration, 
                   columns=["viewpoint_"+str(i+1) for i in range(nViewpoints)])
df_rate_kbps

,viewpoint_1,viewpoint_2,viewpoint_3,viewpoint_4,viewpoint_5
0,0.105182,0.105182,0.105182,0.105182,0.105182
1,0.503977,0.503977,0.503977,0.503977,0.503977
2,1.004706,1.004706,1.004706,1.004706,1.004706
3,2.500472,2.500472,2.500472,2.500472,2.500472
4,4.996337,4.996337,4.996337,4.996337,4.996337
5,7.989850,7.989850,7.989850,7.989850,7.989850
6,15.977459,15.977459,15.977459,15.977459,15.977459
7,34.946454,34.946454,34.946454,34.946454,34.946454


#### Calculater R

* Types of R :
    - lin : q(r_n) = Rn (MPC)
    - Log : q(r_n) = Log(R/R_min)
* Code
    - Input : data rate in kbps
    - Output : q(r_n)

In [119]:
#R(q) types --> lin or log
def Qscore(rate_type):
      #define Q Score
    if (rate_type==0):
        df_qscore=df_rate_kbps
    else:
        Rmin_vp = df_rate_kbps.iloc[0].values
        Rn_vp = np.ones(nViewpoints)
        for vp in range(1,nRates) :
            Rn_vp = np.vstack((Rn_vp, np.log2(df_rate_kbps.iloc[vp].values / Rmin_vp)+1) )
        df_qscore = pd.DataFrame(Rn_vp, columns=df_rate_kbps.columns)
    return df_qscore


#get R(q)
def q(df_qscore,viewpoint, rIndex) :
    return df_qscore.iloc[rIndex, viewpoint-1]

Qscore(1)

,viewpoint_1,viewpoint_2,viewpoint_3,viewpoint_4,viewpoint_5
0,1.000000,1.000000,1.000000,1.000000,1.000000
1,3.260469,3.260469,3.260469,3.260469,3.260469
2,4.255812,4.255812,4.255812,4.255812,4.255812
3,5.571239,5.571239,5.571239,5.571239,5.571239
4,6.569909,6.569909,6.569909,6.569909,6.569909
5,7.247207,7.247207,7.247207,7.247207,7.247207
6,8.247004,8.247004,8.247004,8.247004,8.247004
7,9.376112,9.376112,9.376112,9.376112,9.376112


#### Read Playback

In [120]:
def read_playback(simId, clientId):
    playfilename = csv_path + "playback_sim" + str(simId) +"_cl" + str(clientId) + ".csv"
    df_play = pd.read_csv(playfilename, sep="\t")
    # df_play.head()
    return df_play


In [121]:
def QoE_Rate(df_play,df_qscore, vp, c_alpha, c_mu, c_lambda):
    Vn_list = df_play.vp.values
    Rn_list = df_play.iloc[:,4+vp].values
    Tn_list = (np.diff(df_play.Start) - segDuration)/segDuration
    
    #get Q score
    qscores = np.zeros(Rn_list.size)
    for i in range(Rn_list.size) :
        qscores[i] = q (df_qscore,vp+1, Rn_list[i])


    qoe_part1 = sum((Vn_list == vp) * qscores) + c_alpha * sum((Vn_list != vp) * qscores)
    qoe_part2 = sum((df_play.vp == vp)[1:] * Tn_list) \
                + c_alpha * sum((df_play.vp != vp)[1:] * Tn_list)    
    qoe_part3 = sum((df_play.vp == vp)[1:] * abs(np.diff(qscores))) \
                + c_alpha * sum((df_play.vp != vp)[1:] * abs(np.diff(qscores)))    
    q_total = qoe_part1 - c_mu * qoe_part2 - c_lambda * qoe_part3
    # outstr = "[{}] QoE Total = {:>7.3f}, Part1 = {:>7.3f}, Part 2 = {:>7.3f}, Part 3 = {:>7.3f}".format(vp, \
    #             q_total, qoe_part1, qoe_part2, qoe_part3)
    outstr = "{:>7.3f},{:>7.3f}, {:>7.3f},{:>7.3f}".format( q_total, qoe_part1, qoe_part2, qoe_part3)

    print (outstr)
    return q_total

In [125]:
def QoE_Vmaf(df_play,df_qscore, vp, c_alpha, c_mu, c_lambda):
    Vn_list = df_play.vp.values
    Rn_list = df_play.iloc[:,9+vp].values
    Tn_list = (np.diff(df_play.Start) - segDuration)/segDuration
    
    #get Q score
    qscores = np.zeros(Rn_list.size)
    for i in range(Rn_list.size) :
        qscores[i] = Rn_list[i]

    qoe_part1 = sum((Vn_list == vp) * qscores) + c_alpha * sum((Vn_list != vp) * qscores)
    qoe_part2 = sum((df_play.vp == vp)[1:] * Tn_list) \
                + c_alpha * sum((df_play.vp != vp)[1:] * Tn_list)    
    qoe_part3 = sum((df_play.vp == vp)[1:] * abs(np.diff(qscores))) \
                + c_alpha * sum((df_play.vp != vp)[1:] * abs(np.diff(qscores)))    
    q_total = qoe_part1 - c_mu * qoe_part2 - c_lambda * qoe_part3
    outstr = "[{}] QoE Total = {:>7.3f}, Part1 = {:>7.3f}, Part 2 = {:>7.3f}, Part 3 = {:>7.3f}".format(vp, \
                q_total, qoe_part1, qoe_part2, qoe_part3)
    # outstr = "{:>7.3f},{:>7.3f}, {:>7.3f},{:>7.3f}".format( q_total, qoe_part1, qoe_part2, qoe_part3)
    print (outstr)
    return q_total

#### Calculate delay

In [126]:
def Delay():
    #delay
    Vn_list = df_play.vp.values
    Play_list = [df_play.iloc[:,4:].values[x][Vn_list[x]] for x in range(0,Vn_list.size)]

    x = False
    prev = 0
    Dn_List = []
    for i in range(len(Vn_list)):
        if Vn_list[i] != prev:
            x = True
        if x:
            if Play_list[i] != 0:
                x = False
    
        Dn_List.append(x)
        prev = Vn_list[i]
    Dn_List = np.array(Dn_List)
    return sum((Dn_List)*2)

#### Run

In [131]:
simId = 0
clientId = 0
c_alpha = 0.1
c_lambda = 1.0 #variation pynalty
rate_type=2 #rate=0; log=1; 2=VMAF
c_mu=1.0 #rebuffering pinalty

# if (rate_type==0 ):
#     c_mu = 35 #highest bitrate pinalty for rate-based [MPC]
# elif(rate_type==2):
#     c_mu=20 #?
# else:
#     c_mu = 2.66 # pinalty for MPC-Log [PENSIEVE]

df_play = read_playback(simId,clientId)
df_qscore = Qscore(rate_type)
multi_qoe = 0
for vp in range(nViewpoints) :
    if(rate_type!=2):
        multi_qoe += QoE_Rate(df_play,df_qscore, vp, c_alpha, c_mu, c_lambda)
    else:
        multi_qoe += QoE_Vmaf(df_play,df_qscore, vp, c_alpha, c_mu, c_lambda)


     
print("Multi-View Video QoE {:>7.3f}".format(multi_qoe))

vDelay = Delay()
print("delay = ", vDelay)
QoE_score=multi_qoe-(c_mu*vDelay)

print("Multi-View Video QoE Delay {:>7.3f}".format(QoE_score))


[0] QoE Total = 610.700, Part1 = 682.200, Part 2 =   0.000, Part 3 =  71.500
[1] QoE Total = 231.400, Part1 = 252.600, Part 2 =   0.000, Part 3 =  21.200
[2] QoE Total = 246.600, Part1 = 326.900, Part 2 =   0.000, Part 3 =  80.300
[3] QoE Total =  58.600, Part1 =  81.000, Part 2 =   0.000, Part 3 =  22.400
[4] QoE Total = 113.800, Part1 = 189.000, Part 2 =   0.000, Part 3 =  75.200
Multi-View Video QoE 1261.100
delay =  4
Multi-View Video QoE Delay 1257.100
